In [2]:
from gurobipy import *
import pandas as pd
from random import seed
from random import gauss
seed(100)

# Battery Sizing and Prices

State the technical parameters of the battery, including size and efficiency. Sample electricity prices from a gaussian distribution, assuming a perfect forecast of prices. These will be the parameters of the optimisation problem.

In [3]:
m=Model('Deterministic_Battery_Scheduling')
#add parameters
Cinitial=53 #initial charge (MWh)
Cmin=0.5 #minimum allowed battery charge level (MWh)
Cmax=53 #maximum allowed battery charge level (MWh)
Kappa=1.1 #battery discharge efficiency
Rho=0.95 #battery charge efficiency
Gamma=100 #battery charge-discharge cycle parameter
Charge_max=20 #maximum charge rate (MW)
Discharge_max=20 #maximum discharge rate (MW)

#create dimensions of the problem, 48 time periods chosen as imbalance prices set at HH granularity
P='period'
Periods=[i for i in range(48)]

Price_DA={}
Price_Imb={}

for i in Periods:
    Price_DA[i]=gauss(50, 2)
    Price_Imb[i]=gauss(45,25)
    



Restricted license - for non-production use only - expires 2025-11-24


# Variables

Create the variables of the optimisation problem

In [4]:
#instantiate variables
generation = {}
demand = {}
dec_DA = {}
generation_upward = {}
generation_downward = {}
dec_supply_Imb = {}
dec_demand_Imb = {}
demand_downward = {}
demand_upward = {}
dec_supply_to_demand = {}
dec_demand_to_supply = {}
special_case_demand_upward = {}
special_case_generation_upward = {}
ending_charge = {}


#create decision variables
#units of energy supplied in DA Market (MW)
for i in Periods:
    generation[i] = m.addVar(lb = 0, ub = Discharge_max, vtype = GRB.CONTINUOUS, name = 'generation_%s'%(i))
    
    #units of energy consumed in DA Market (MW)
    demand[i] = m.addVar(lb = 0, ub = Charge_max, vtype = GRB.CONTINUOUS, name = 'demand_%s'%(i))
    
    #decision variable at DA market, when equal to 1 energy is supplied, when equal to 0 energy is consumed in that period
    dec_DA[i] = m.addVar(vtype=GRB.BINARY, name = 'dec_DA_%s'%(i))
    
    #when supplying, increase output in Balancing Market
    generation_upward[i] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'generation_upward_%s'%(i))

    #when supplying, decrease output in Balancing Market
    generation_downward[i] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'generation_downward_%s'%(i))

    #when supplying in DA decision variable at Imb market to increase(1) or decrease(0) output 
    dec_supply_Imb[i] = m.addVar(vtype=GRB.BINARY, name = 'dec_supply_Imb_%s'%(i))

    #when consuming, increase consumption in Balancing Market
    demand_upward[i] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'demand_upward_%s'%(i))

    #when consuming, decrease consumption in Balancing Market
    demand_downward[i] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'demand_downward_%s'%(i))

    #when consuming in DA decision variable at Imb market to increase(1) or decrease(0) output 
    dec_demand_Imb[i] = m.addVar(vtype=GRB.BINARY, name = 'dec_demand_Imb_%s'%(i))
    
    #special case: when set to supply in DA, switch to consuming in balancing market
    special_case_demand_upward[i]=m.addVar(lb=0, vtype=GRB.CONTINUOUS, name='special_case_demand_upward_%s'%(i))
    
    #when supplying in DA switch to consuming decision variable
    dec_supply_to_demand[i]=m.addVar(vtype=GRB.BINARY, name='dec_supply_to_demand_%s'%(i))

    #special case: when set to consume in DA, switch to supplying in balancing market
    special_case_generation_upward[i]=m.addVar(lb=0, vtype=GRB.CONTINUOUS, name='special_case_generation_upward_%s'%(i))
    
    #when consuming in DA switch to supply decision variable
    dec_demand_to_supply[i]=m.addVar(vtype=GRB.BINARY, name='dec_demand_to_supply_%s'%(i))
    
    #battery charge at the end of each period
    ending_charge[i]=m.addVar(lb=Cmin, ub=Cmax, vtype=GRB.CONTINUOUS, name='initial_charge_%s'%(i))

m.update()


# Constraints and Objective

Set the constraints and objective of the optimisation problem

In [5]:
#set the objective, expected profit maximization
obj1 = quicksum(Price_DA[i]*(0.5*generation[i]-0.5*demand[i]) for i in Periods )
obj2 = quicksum(Price_Imb[i]*(0.5*generation_upward[i]+0.5*demand_downward[i]+0.5*special_case_generation_upward[i]-0.5*generation_downward[i]-0.5*demand_upward[i]-0.5*special_case_demand_upward[i]) for i in Periods )
obj = obj1 + obj2
m.setObjective(obj, GRB.MAXIMIZE)

#define constraints
for i in Periods:
    #rate of discharge before balancing introduced must not exceed max discharge 
    m.addConstr(generation[i]<= (dec_DA[i])*(Discharge_max), 'initial_max_discharge_constr')
    
    #rate of charge before balancing introduced must not exceed max charge 
    m.addConstr(demand[i]<= (1-dec_DA[i])*(Charge_max), 'initial_max_charge_constr')
    
    #rate of discharge in a period must not exceed max discharge rate when set to supply in DA and upward balancing
    m.addConstr(generation[i]+generation_upward[i] <= (dec_DA[i])*(dec_supply_Imb[i])*(Discharge_max), 'max_discharge_constr')
    
    #rate of discharge must not drop below zero when set to supply in DA and downward balancing
    m.addConstr((generation_downward[i]-generation[i]) <= 0, 'min_discharge_constr_1')
    m.addConstr(generation_downward[i] <= (dec_DA[i])*(1-dec_supply_Imb[i])*(Discharge_max), 'min_discharge_constr_2')
    
    #rate of charge in a period must not exceed max charge rate when set to consume in DA and upward balancing
    m.addConstr(demand[i]+demand_upward[i] <= (1-dec_DA[i])*(dec_demand_Imb[i])*(Charge_max), 'max_charge_constr')
                                                          
    #rate of discharge must not drop below zero when set to consume in DA and downward balancing
    m.addConstr((demand_downward[i]-demand[i]) <= 0, 'min_charge_constr_1')
    m.addConstr(demand_downward[i] <= (1-dec_DA[i])*(1-dec_demand_Imb[i])*(Charge_max), 'min_charge_constr_2')
                                                          
    #special case 1: when supplying in DA switch to consuming decision variable (decision variable can only equal one if generation_downward=generation at time period i)
    m.addConstr(dec_supply_to_demand[i] <= (dec_DA[i])*(1-dec_supply_Imb[i]), 'supply_to_consume_dec_constr_1')
    m.addConstr(dec_supply_to_demand[i]*generation[i]<=generation_downward[i], 'supply_to_consume_dec_constr_2')
    
    #special case 1: rate of charge must not exceed max charge rate
    m.addConstr(special_case_demand_upward[i] <= (dec_supply_to_demand[i])*(Charge_max), 'special_case_1_max_charge_constr')        
                
    #special case 2: when consuming in DA switch to supplying decision variable (decision variable can only equal one if demand_downward=demand at time period i)
    m.addConstr(dec_demand_to_supply[i] <= (1-dec_DA[i])*(1-dec_demand_Imb[i]), 'consume_to_supply_dec_constr_1')
    m.addConstr(dec_demand_to_supply[i]*demand[i] <= demand_downward[i], 'consume_to_supply_dec_constr_2')
    
    #special case 2: rate of discharge must not exceed max discharge rate
    m.addConstr(special_case_generation_upward[i] <= (dec_demand_to_supply[i])*(Discharge_max), 'special_case_2_max_discharge_constr') 
    
    #the inital charge of the battery plus the sum of the discharging/charging done during this period plus previous periods must fall within the operating capacity range of the battery, charge and discharge efficiency are also accounted for
    m.addConstr(ending_charge[i] == Cinitial-quicksum(generation[j]+generation_upward[j]-generation_downward[j]+special_case_generation_upward[j] for j in range(i+1))*0.5*Kappa+quicksum(demand[j]+demand_upward[j]-demand_downward[j]+special_case_demand_upward[j] for j in range(i+1))*0.5*Rho, 'initial_charge_constr')
    

#Gamma is a design parameter limiting the total number of daily discharge cycles
m.addConstr(quicksum(generation[i]+generation_upward[i]-generation_downward[i]+special_case_generation_upward[i] for i in Periods) <= Gamma*(Cmax-Cmin))


<gurobi.Constr *Awaiting Model Update*>

# Run Optimisation

In [6]:
m.optimize()
if m.status == GRB.status.OPTIMAL:
    print('\nMaximum objective is :',m.objval)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 18 physical cores, 18 logical processors, using up to 18 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
if m.status == GRB.status.OPTIMAL:
    print ('\nOptimal Scheduling :')
    for i in Periods:
        print('supplying at period'+ str(i) + ': ' + str(generation[i].x))
        print('consuming at period'+ str(i) + ': ' + str(demand[i].x))
        print('upward supply balancing at period'+ str(i) + ': ' + str(generation_upward[i].x))
        print('downward supply balancing at period'+ str(i) + ': ' + str(generation_downward[i].x))
        print('special case upward supply balancing at period'+ str(i) + ': ' + str(special_case_generation_upward[i].x))
        print('upward demand balancing at period'+ str(i) + ': ' + str(demand_upward[i].x))
        print('downward demand balancing at period'+ str(i) + ': ' + str(demand_downward[i].x))
        print('special case upward demand balancing at period'+ str(i) + ': ' + str(special_case_demand_upward[i].x))
        
        

# Save the Results

In [ ]:
results=pd.DataFrame()
results['supply']=pd.Series(generation[i].x for i in Periods)
results['demand']=pd.Series(demand[i].x for i in Periods)
results['supply upward balancing']=pd.Series(generation_upward[i].x for i in Periods)
results['supply downward balancing']=pd.Series(generation_downward[i].x for i in Periods)
results['special case supply upward balancing']=pd.Series(special_case_generation_upward[i].x for i in Periods)
results['demand upward balancing']=pd.Series(demand_upward[i].x for i in Periods)
results['demand downward balancing']=pd.Series(demand_downward[i].x for i in Periods)
results['special case demand upward balancing']=pd.Series(special_case_demand_upward[i].x for i in Periods)
results['dec_DA']=pd.Series(dec_DA[i].x for i in Periods)
results['dec_supply_Imb']=pd.Series(dec_supply_Imb[i].x for i in Periods)
results['dec_demand_Imb']=pd.Series(dec_demand_Imb[i].x for i in Periods)
results['dec_supply_to_demand']=pd.Series(dec_supply_to_demand[i].x for i in Periods)
results['dec_demand_to_supply']=pd.Series(dec_demand_to_supply[i].x for i in Periods)
results['DA Price']=pd.Series(Price_DA[i] for i in Periods)
results['Imb Price']=pd.Series(Price_Imb[i] for i in Periods)
results['Ending Charge']=pd.Series(ending_charge[i].x for i in Periods)


In [ ]:
results.to_excel('Results.xlsx')